In [2]:
!pip install split-folders

In [1]:
import pathlib
import splitfolders
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
import os

C:\Users\aliz\Anaconda3\envs\Time_Series\lib\site-packages\requests\__init__.py:104: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  RequestsDependencyWarning)
C:\Users\aliz\Anaconda3\envs\Time_Series\lib\site-packages\requests\__init__.py:104: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  RequestsDependencyWarning)


In [2]:
import pathlib
Dataset_dir = r'D:\Mubassira\KaggleProject\Pistacho\Pistachio_Image_Dataset\Pistachio_Image_Dataset'
Dataset_dir =pathlib.Path(Dataset_dir)

splitfolders.ratio(Dataset_dir, output="Pistachio_dataset", seed=101, ratio=(.7, .2, .1))

Copying files: 2148 files [00:02, 981.03 files/s] 


In [3]:
Dataset_dir 


WindowsPath('D:/Mubassira/KaggleProject/Pistacho/Pistachio_Image_Dataset/Pistachio_Image_Dataset')

In [4]:
## Shape of the image

In [5]:
# We can check image shape using this code
import cv2
import numpy as np

# Load the image
image = cv2.imread('../Pistachio_Image_Dataset/Pistachio_dataset/train/Kirmizi_Pistachio/kirmizi (12).jpg')

# Check the shape of the image
height, width, channels = image.shape
# Dataset comprises of images of shape: (600,600,3)
# Print the image shape
print(f"Image shape: (height={height}, width={width}, channels={channels})")

Image shape: (height=600, width=600, channels=3)


In [6]:
training_dir = "../Pistachio_Image_Dataset/Pistachio_dataset/train"
test_dir = "../Pistachio_Image_Dataset/Pistachio_dataset/test"
val_dir = "../Pistachio_Image_Dataset/Pistachio_dataset/val"

In [14]:


# Display the image in a window with the name "image"
cv2.imshow("image", image)
# Wait for a key event and close the window when a key is pressed
cv2.waitKey(0)
cv2.destroyAllWindows()

##
Data augmentation is a technique used to artificially increase the diversity of your training dataset by applying various transformations to the existing images. This helps improve the generalization and robustness of a machine learning model. Let's go through each parameter in your code:

In [7]:
# Define image size and batch size
image_height, image_width = 224, 224
batch_size = 32

# Data Augmentation for Training
train_datagen = ImageDataGenerator(
    rescale=1./255,
    brightness_range=[0.8, 1.2],
    width_shift_range=0.2,
    height_shift_range=0.2,
    rotation_range=20,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Validation Data Generator (No Data Augmentation)
val_datagen = ImageDataGenerator(rescale=1./255)

### Load and Preprocess Data:
* Load your dataset and preprocess it. 
* If you are working with image data, you might use the ImageDataGenerator for data augmentation and preprocessing.

In [9]:
# Training Data Generator
# Training Data Generator
train_generator = train_datagen.flow_from_directory(
    training_dir,
    target_size=(image_height, image_width),
    batch_size=batch_size,
    class_mode='categorical',
    classes=['Kirmizi_Pistachio', 'Siirt_Pistachio']  # Specify class labels
)

# Validation Data Generator

val_datagen = ImageDataGenerator(
    rescale=1./255
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(image_height, image_width),
    batch_size=batch_size,
    class_mode='categorical',
    classes=['Kirmizi_Pistachio', 'Siirt_Pistachio']  # Specify class labels
)

Found 1503 images belonging to 2 classes.
Found 429 images belonging to 2 classes.


In [10]:

# We have to created image generator for the validation dataset


# For testing

test_datagen = ImageDataGenerator(
    rescale=1./255
    )
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(image_height, image_width),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle = True,
    classes=['Kirmizi_Pistachio', 'Siirt_Pistachio']  # Specify class labels
)

Found 216 images belonging to 2 classes.


In [23]:
 ## After the dataset are seperated,we need to make the label file according to each file

In [15]:
import pandas as pd
import glob

Total_TrainImages = glob.glob('../Pistachio_Image_Dataset/Pistachio_dataset/train/*/*.jpg')
print("Total number of training images: ", len(Total_TrainImages))


Total number of training images:  1503


In [11]:
# Get a batch of images and labels
images, labels = next(train_generator)

In [13]:
# Get a batch of images and labels
images, labels = next(train_generator)

# Print the shape of the batch
print("Batch shape - Images:", images.shape)
print("Batch shape - Labels:", labels.shape)

# Print the labels for the first few images
for i in range(min(batch_size, 5)):  # Print labels for the first 5 images
    print(f"Image {i+1}: Label - {labels[i]}")

Batch shape - Images: (32, 224, 224, 3)
Batch shape - Labels: (32, 2)
Image 1: Label - [1. 0.]
Image 2: Label - [0. 1.]
Image 3: Label - [1. 0.]
Image 4: Label - [1. 0.]
Image 5: Label - [0. 1.]


In [18]:
# Initialize lists to store file paths and labels
file_paths = []
labels = []

# Specify the number of batches to visualize (adjust as needed)
num_batches = 5

# Iterate through the generator and accumulate file paths and labels
for _ in range(num_batches):
    batch_images, batch_labels = next(train_generator)
    batch_file_paths = train_generator.filepaths  # Get the file paths from the generator

    file_paths.extend(batch_file_paths)
    labels.extend(batch_labels)

# Convert lists to a Pandas DataFrame
df = pd.DataFrame({
    'File_Path': file_paths[:len(labels)],  # Ensure both lists have the same length
    'Label': np.argmax(labels, axis=1)  # Convert one-hot encoded labels to class indices
})



In [19]:
df.head()

,File_Path,Label
0,../Pistachio_Image_Dataset/Pistachio_dataset/t...,0
1,../Pistachio_Image_Dataset/Pistachio_dataset/t...,1
2,../Pistachio_Image_Dataset/Pistachio_dataset/t...,0
3,../Pistachio_Image_Dataset/Pistachio_dataset/t...,0
4,../Pistachio_Image_Dataset/Pistachio_dataset/t...,0


In [20]:
df

,File_Path,Label
0,../Pistachio_Image_Dataset/Pistachio_dataset/t...,0
1,../Pistachio_Image_Dataset/Pistachio_dataset/t...,1
2,../Pistachio_Image_Dataset/Pistachio_dataset/t...,0
3,../Pistachio_Image_Dataset/Pistachio_dataset/t...,0
4,../Pistachio_Image_Dataset/Pistachio_dataset/t...,0
...,...,...
155,../Pistachio_Image_Dataset/Pistachio_dataset/t...,0
156,../Pistachio_Image_Dataset/Pistachio_dataset/t...,0
157,../Pistachio_Image_Dataset/Pistachio_dataset/t...,0
158,../Pistachio_Image_Dataset/Pistachio_dataset/t...,0


In [31]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense


model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(image_height, image_width, 3)))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(2, activation='softmax'))  # Assuming 2 classes: Kirmizi_Pistachio and Siirt_Pistachio
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [32]:
num_epochs = 5  # You can adjust this based on your needs

model.fit(
    train_generator,
    epochs=num_epochs,
    validation_data=val_generator
)


Epoch 1/5
47/47 [==============================] - 52s 1s/step - loss: 0.8219 - accuracy: 0.5462 - val_loss: 0.6897 - val_accuracy: 0.5734
Epoch 2/5
47/47 [==============================] - 46s 980ms/step - loss: 0.6537 - accuracy: 0.6148 - val_loss: 0.6087 - val_accuracy: 0.7156
Epoch 3/5
47/47 [==============================] - 43s 918ms/step - loss: 0.5965 - accuracy: 0.6833 - val_loss: 0.5552 - val_accuracy: 0.6597
Epoch 4/5
47/47 [==============================] - 43s 916ms/step - loss: 0.6492 - accuracy: 0.6228 - val_loss: 0.6025 - val_accuracy: 0.6830
Epoch 5/5
47/47 [==============================] - 44s 938ms/step - loss: 0.5879 - accuracy: 0.6919 - val_loss: 0.5234 - val_accuracy: 0.7949


In [33]:
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_generator)
print(f"Test Accuracy: {test_accuracy}")

7/7 [==============================] - 1s 186ms/step - loss: 0.5399 - accuracy: 0.7824
Test Accuracy: 0.7824074029922485
